In [4]:
import osmnx as ox
import geopandas as gpd
import folium
import random
from shapely.geometry import Point

# 1. Define the area
place = "Ashoka Colony, Hanamkonda, Telangana, India"

# 2. Get building footprints
buildings = ox.features_from_place(place, tags={"building": True})
buildings = buildings[buildings.geometry.type == "Polygon"]
buildings = buildings.to_crs(epsg=4326)

# 3. Geocode a named place to get base point (post office)
base_name = "Hanamkonda Head Post Office, Hanamkonda, Telangana, India"
lat, lon = ox.geocode(base_name)  # returns (lat, lon)
base_point = Point(lon, lat)
base_gdf = gpd.GeoDataFrame(geometry=[base_point], crs="EPSG:4326")


# 4. Create the folium map centered on the base
map_center = [lat, lon]
m = folium.Map(location=map_center, zoom_start=18)

# 5. Draw buildings
for _, row in buildings.iterrows():
    geom = row.geometry
    if geom.geom_type == "Polygon":
        folium.Polygon(
            locations=[(y, x) for x, y in geom.exterior.coords],
            color='gray', weight=0.5, fill=True, fill_opacity=0.3
        ).add_to(m)

# 6. Add the drone base marker
folium.Marker(
    location=[lat, lon],
    popup=f"Drone Base: {base_name}",
    icon=folium.Icon(color='green', icon='home')
).add_to(m)


# 7. Show the map
m

In [ ]:
import osmnx as ox
import folium
from shapely.geometry import Point
import geopandas as gpd
import pandas as pd


# Create a 5000-meter buffer around the base point
base_buffer = gpd.GeoSeries([base_point], crs="EPSG:4326").to_crs(epsg=3857).buffer(5000).to_crs(epsg=4326)
area_polygon = base_buffer.geometry.iloc[0]

# Define tags for restricted zones, corrected for OSM standards
tags = {
    # Commercial areas
    "shop": ["mall", "supermarket", "department_store"],
    "building": ["retail", "temple"],  # Added temple as a building type

    # Health zones
    "amenity": ["hospital", "clinic", "doctors", "school", "college", "university", "place_of_worship"],  # Removed temple
    "healthcare": ["hospital", "clinic", "doctor"],

    # Sensitive zones
    "landuse": ["military"],
    "aeroway": ["aerodrome", "helipad"]
}

# Fetch features from OSM with debugging output
found = []
for key, vals in tags.items():
    for val in vals:
        try:
            gdf = ox.features_from_polygon(area_polygon, tags={key: val})
            if not gdf.empty:
                gdf["zf_type"] = f"{key}={val}"
                found.append(gdf.to_crs(epsg=4326))
            else:
                print(f"No features found for {key}={val}")
        except Exception as e:
            print(f"Error querying {key}={val}: {str(e)}")

# Combine found features into a single GeoDataFrame
nfz = gpd.GeoDataFrame(pd.concat(found, ignore_index=True), crs="EPSG:4326") if found else gpd.GeoDataFrame(columns=["geometry"], crs="EPSG:4326")

# Create a Folium map centered on the base point
m = folium.Map(location=[lat, lon], zoom_start=15)

# Add the buffer zone to the map for reference
folium.GeoJson(
    base_buffer.__geo_interface__,
    style_function=lambda f: {
        "fillColor": "blue",
        "color": "blue",
        "fillOpacity": 0.1,
        "weight": 1
    },
    tooltip="Buffer Zone (1000m)"
).add_to(m)

# Add the base point to the map
folium.Marker(
    location=[lat, lon],
    popup="Base Point",
    icon=folium.Icon(color="green", icon="home")
).add_to(m)

# Add restricted zones to the map
for _, zone in nfz.iterrows():
    geom = zone.geometry
    ztype = zone.get("zf_type", "Restricted Zone")
    name = zone.get("name", "")
    label = f"{name} ({ztype})" if name else ztype

    try:
        if geom.geom_type == "Point":
            folium.CircleMarker(
                location=[geom.y, geom.x],
                radius=6,
                color="red",
                fill=True,
                fill_opacity=0.7,
                tooltip=label
            ).add_to(m)
        elif geom.geom_type in ["Polygon", "MultiPolygon"]:
            folium.GeoJson(
                geom,
                style_function=lambda f: {
                    "fillColor": "red",
                    "color": "darkred",
                    "fillOpacity": 0.5,
                    "weight": 1
                },
                tooltip=label
            ).add_to(m)
        else:
            print(f"Unsupported geometry type for {label}: {geom.geom_type}")
    except Exception as e:
        print(f"Error rendering {label}: {str(e)}")

# Display the map
m

Error querying shop=department_store: No matching features. Check query location, tags, and log.
Error querying building=temple: No matching features. Check query location, tags, and log.
Error querying landuse=military: No matching features. Check query location, tags, and log.
Error querying aeroway=aerodrome: No matching features. Check query location, tags, and log.
Error querying aeroway=helipad: No matching features. Check query location, tags, and log.


In [6]:
import requests
import folium

def get_geocode_earth_coords(location_name, api_key):
    url = f"https://api.geocode.earth/v1/search?text={location_name}, Telangana, India&api_key={api_key}&size=5"

    try:
        res = requests.get(url)
        data = res.json()

        results = data.get("features", [])
        if not results:
            print("❌ No matching locations found.")
            return None

        print("\n🔍 Found the following locations:")
        for idx, feature in enumerate(results):
            label = feature['properties'].get('label', 'Unknown')
            print(f"{idx + 1}. {label}")

        choice = int(input("\n👉 Choose location number to select (1–{}): ".format(len(results))))
        if 1 <= choice <= len(results):
            coords = results[choice - 1]['geometry']['coordinates']
            return coords[1], coords[0], results[choice - 1]['properties']['label']
        else:
            print("❌ Invalid choice.")
            return None

    except Exception as e:
        print(f"Error: {e}")
        return None

# ✨ User Input + Geocoding + Folium Plotting
api_key = "ge-84b12eb50c43dc6d"  # 🔑 Insert your Geocode Earth API key
user_input = input("📍 Enter delivery location (e.g., Hanamkonda Bus Stand): ")

result = get_geocode_earth_coords(user_input, api_key)
if result:
    lat, lon, label = result
    print(f"\n✅ Selected: {label} → Coordinates: {lat}, {lon}")


    folium.Marker(
        location=(lat, lon),
        popup=f"📦 Delivery Point: {label}",
        icon=folium.Icon(color='blue', icon='cloud')
    ).add_to(m)

else:
    print("❌ Could not plot the delivery location.")


❌ No matching locations found.
❌ Could not plot the delivery location.


In [7]:
m

In [8]:
nfz.to_file("nfz.geojson", driver="GeoJSON")